In [76]:
import numpy as np

In [77]:
def apply_simulation_parameters_mutation(individual, mutation_rate, means, std_devs, min_vals, max_vals, distribution):
 
    mut_mask = np.random.rand(2) < mutation_rate
    
    for i in range(2):
        
        if distribution == "uniform":
            individual[-1, -1, i+3] += (np.random.uniform(low=min_vals[i], high=max_vals[i]) - individual[-1, -1, i+3]) * mut_mask[i]
        elif distribution == "normal":
            individual[-1, -1, i+3] += np.random.normal(loc=means[i], scale=std_devs[i]) * mut_mask[i]
        
        individual[-1, -1, i+3] = max(min_vals[i], min(max_vals[i], individual[-1, -1, i+3]))

    return individual


In [78]:
for i in range(20):
    ind = np.zeros((3, 10, 10))
    ind[-1, -1, 3:5] = [20, .2]
    ind = apply_simulation_parameters_mutation(ind, 0.9, [10, .1], [5, .01], [5, .00001], [100, .1], "uniform")
    print(ind[-1, -1, :])

[ 0.          0.          0.         37.52759172  0.05441912  0.
  0.          0.          0.          0.        ]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 4.30413463e+01
 3.28390036e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 7.28250926e+00
 7.24989972e-03 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
[ 0.          0.          0.         89.81335426  0.1         0.
  0.          0.          0.          0.        ]
[ 0.          0.          0.         17.34162218  0.04184204  0.
  0.          0.          0.          0.        ]
[ 0.          0.          0.         20.          0.02352531  0.
  0.          0.          0.          0.        ]
[0.00000000e+00 0.00000000e+00 0.00000000e+00 6.85644611e+01
 5.46912956e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00]
[ 0.          0.          0.         20.          0.06251275  0.
  0.         

In [111]:
def apply_compartment_mutation(individual, mutation_rate, mean, std_dev, min_val, max_val, distribution):

    num_species = int(individual[-1, -1, 0])
    z, y, x = individual.shape
   
    for i in range(1, num_species * 2, 2):
        mut_mask = np.random.rand(y, x) < mutation_rate

        if distribution == "normal":
            noise = np.random.normal(loc=mean, scale=std_dev, size=(y, x))
        elif distribution == "uniform":
            noise = np.random.uniform(low=min_val, high=max_val, size=(y, x))

        individual[i, :, :] += np.where(mut_mask, noise, 0)
        individual[i, :, :] = np.clip(individual[i, :, :], min_val, max_val)

    return individual


In [112]:
ind = np.zeros((4, 5, 5))
ind[-1, -1, 0] = 2

In [113]:
ind = apply_compartment_mutation(ind, .3, 10, 3, 0, 100, "normal")

In [114]:
ind

array([[[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]],

       [[14.0717607 ,  0.        , 10.37036457,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
         16.64442524],
        [10.76495102,  0.        , 10.53249029,  0.        ,
          0.        ],
        [ 0.        , 10.38751566,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  9.13504115,  0.        ,
          9.68256362]],

       [[ 0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,
          0

In [137]:
def apply_parameter_mutation(individual, mutation_rate, species_means, species_std_devs, species_min_vals, species_max_vals, 
                             complex_means, complex_std_devs, complex_min_vals, complex_max_vals, distribution):

    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    pair_start = num_species * 2
    pair_stop = pair_start + (num_pairs * 2)

    count = 0
    for i in range(0, num_species * 2, 2):
        mut_mask = np.random.rand(3) < mutation_rate
        if distribution == "normal":
            for j in range(3):
                individual[-1, i, j] += np.random.normal(loc=species_means[count], scale=species_std_devs[count]) * mut_mask[j]
        elif distribution == "uniform":
            for j in range(3):
                individual[-1, i, j] += (np.random.uniform(low=species_min_vals[count], high=species_max_vals[count]) - individual[-1, i, j]) * mut_mask[j]
        count += 1
        
    for i in range(pair_start+1, pair_stop, 2):
        mut_mask = np.random.rand(4) < mutation_rate
        if distribution == "normal":
            for j in range(4):
                individual[i, 1, j] += np.random.normal(loc=complex_means[count], scale=complex_std_devs[count]) * mut_mask[j]
        elif distribution == "uniform":
            for j in range(4):
                individual[i, 1, j] += (np.random.uniform(low=complex_min_vals[count], high=complex_max_vals[count]) - individual[i, 1, j]) * mut_mask[j]
        count += 1

    return individual

In [138]:
ind = np.zeros((7, 10, 10))
ind[-1, 0, :3] = [.2, .5, .7]
ind[-1, 2, :3] = [.2, .5, .7]
ind[-2, 1, :4] = [20, 10, 30, 81]

In [139]:
for i in range(20):
    ind = np.zeros((9, 5, 5))
    ind[-1, -1, 0] = 2
    ind[-1, -1, 1] = 2
    ind[-1, 0, :3] = [.2, .5, .7]
    ind[-1, 2, :3] = [.2, .5, .7]
    ind[-4, 1, :4] = [20, 10, 30, 80]
    ind[-2, 1, :4] = [20, 10, 30, 80]
    ind = apply_parameter_mutation(ind, .9, [1, 1, 1], [.2, .2, .3], [0, 0, 0], [5, 5, 5], 
                                   [3,4,5,5], [1,1,1,1], [0, 0, 0, 0], [10,10,10,10],"uniform")
    print(ind[-2:, :,:])

[[[ 0.          0.          0.          0.          0.        ]
  [ 3.62373921 10.          6.43852797  3.88023318  0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]]

 [[ 3.23656519  0.27588055  1.3067239   0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 0.59899239  1.21898785  0.98568723  0.          0.        ]
  [ 0.          0.          0.          0.          0.        ]
  [ 2.          2.          0.          0.          0.        ]]]
[[[0.         0.         0.         0.         0.        ]
  [8.12369316 5.90068889 8.00768263 2.07185333 0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]
  [0.         0.         0.         0.         0.        ]]

 [[3.52875232 4.71259768 1.11572155 0.         0.        ]


In [183]:
import itertools

def apply_species_insertion_mutation(individual, mutation_rate):
    """
    Applies a species insertion mutation to the individual with a given probability.

    This function adds a new species to the individual if a random value is below the specified mutation rate.
    When a new species is added, it automatically generates all possible complexes between the new species and 
    the existing species. The updated individual structure is then returned.

    Parameters:
    - individual (numpy.ndarray): The multi-dimensional array representing the species and complexes.
    - mutation_rate (float): The probability of adding a new species.

    Returns:
    - numpy.ndarray: The updated individual with a new species and its complexes if the mutation occurred.
    """

    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    z, y, x = individual.shape

    if np.random.rand() < mutation_rate:
        pairs = pair_finding(num_species=num_species)
        init_matrix = species_initialization(compartment_size=(y, x), pairs=pairs)
        individual = species_combine(individual=individual, init_matrix=init_matrix, num_species=num_species, num_pairs=num_pairs)
    
    return individual

def pair_finding(num_species):
    """
    Finds all possible pairs between the new species and the existing species.

    Given the current number of species, this function generates all possible pairs between 
    the new species (which is one more than the current number) and the existing species.

    Parameters:
    - num_species (int): The current number of species.

    Returns:
    - list of tuples: A list of pairs (as tuples) where each pair includes the new species.
    """

    last = num_species + 1
    species = [i for i in range(1, num_species + 2, 1)]
    pairs = list(itertools.combinations(species, 2))

    out_pairs = [pair for pair in pairs if last in pair]

    return out_pairs

def species_initialization(compartment_size, pairs):
    """
    Initializes the parameters for the new species and its complexes.

    This function creates an initialization matrix containing the new species and the complexes 
    formed between the new species and each existing species. It sets random initial values for the 
    parameters of these species and complexes.

    Parameters:
    - compartment_size (tuple of int): The size of each compartment in the individual matrix.
    - pairs (list of tuples): A list of pairs representing the complexes between the new species and existing species.

    Returns:
    - numpy.ndarray: A matrix containing the initialized values for the new species and its complexes.
    """

    num_species = len(pairs) + 1
    num_matrices = num_species * 2
    init_matrix = np.zeros((num_matrices, compartment_size[0], compartment_size[1]))

    for i in range(len(pairs)):
        m = np.zeros((2, compartment_size[0], compartment_size[1]))
        m[-1, 0, 0] = int(pairs[i][0])
        m[-1, 0, 1] = int(pairs[i][1])
        m[-1, 1, :4] = np.random.rand(4)
        init_matrix[i*2+2:i*2+4, :, :] = m

    return init_matrix

def species_combine(individual, init_matrix, num_species, num_pairs):
    """
    Combines the new species and its complexes with the existing individual.

    This function takes the existing individual matrix and the initialization matrix for the new species and its 
    complexes, and combines them into a single updated individual matrix. It also updates the metadata to reflect 
    the addition of the new species and complexes.

    Parameters:
    - individual (numpy.ndarray): The original individual matrix.
    - init_matrix (numpy.ndarray): The initialization matrix for the new species and complexes.
    - num_species (int): The original number of species.
    - num_pairs (int): The original number of complexes.

    Returns:
    - numpy.ndarray: The updated individual matrix with the new species and complexes added.
    """

    z, y, x = individual.shape
    z1 = z + init_matrix.shape[0]

    updated_individual = np.zeros((z1, y, x))
    updated_individual[:num_species*2, :, :] = individual[:num_species*2, :, :]
    updated_individual[num_species*2:num_species*2+init_matrix.shape[0], :, :] = init_matrix
    updated_individual[num_species*2+init_matrix.shape[0]:, :, :] = individual[num_species*2:, :, :]
    updated_individual[-1, -1, 0] = int(num_species+1)
    updated_individual[-1, -1, 1] = int(num_pairs+((init_matrix.shape[0]-2)/2))
    updated_individual[-1, num_species*2, :3] = np.random.rand(3)

    return updated_individual

In [184]:
ind = np.zeros((7, 7, 7))
ind[-1, -1, 0] = 2
ind[-1, -1, 1] = 1
ind[5, 0, :2] = [0, 2]
ind[-1, 0, :3] = [.2, .5, .7]
ind[-1, 2, :3] = [.2, .5, .7]
ind[-2, 1, :4] = [20, 10, 30, 80]
print(ind)
ind = apply_species_insertion_mutation(ind, .9)

[[[ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]]

 [[ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]]

 [[ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]]

 [[ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   0.   0.   0.   0. ]
  [ 0.   0.   0.   

In [185]:
ind

array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]],

       [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        

In [230]:
def apply_species_deletion_mutation(individual, mutation_rate):

    num_species = int(individual[-1, -1, 0])
    if np.random.rand() < mutation_rate:
        deleted_species = int(np.random.choice(np.arange(1, num_species)))
        individual = species_deletion(individual=individual, deleted_species=deleted_species)

    return individual






def species_deletion(individual, deleted_species):

    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    pair_start = int((num_species * 2) + 1)
    pair_stop = int(pair_start + (num_pairs * 2))
    count = [deleted_species*2, deleted_species*2+1]

    for i in range(pair_start, pair_stop, 2):
        if int(individual[i, 0, 0]) == int(deleted_species) or int(individual[i, 0, 1]) == int(deleted_species):
            count.append(i-1)
            count.append(i)
    updated_individual = np.delete(individual, count, axis=0)

    return updated_individual

In [231]:
ind = np.zeros((7, 10, 10))
ind[-1, -1, 0] = 2
ind[-1, -1, 1] = 1
ind[5, 0, :2] = [0, 2]
ind[-1, 0, :3] = [.2, .5, .7]
ind[-1, 2, :3] = [.2, .5, .7]
ind[-2, 1, :4] = [20, 10, 30, 80]
print(ind.shape)
ind = apply_species_insertion_mutation(ind, .9)
print(ind.shape)
ind = apply_species_deletion_mutation(ind, .95)
print(ind.shape)

(7, 10, 10)
(13, 10, 10)
(9, 10, 10)


In [196]:
n = np.random.choice(np.arange(1, 3))
n

2

In [244]:
def species_deletion(individual, deleted_species):
    """
    Deletes a species and all complexes involving that species from the individual matrix.

    Parameters:
    - individual (numpy.ndarray): The individual matrix before deletion.
    - deleted_species (int): The index of the species to be deleted.

    Returns:
    - numpy.ndarray: The updated individual matrix after deletion.
    """
    num_species = int(individual[-1, -1, 0])
    num_pairs = int(individual[-1, -1, 1])
    pair_start = int((num_species * 2) + 1)
    pair_stop = int(pair_start + (num_pairs * 2))
    
    # Indices to delete: species and complexes involving the deleted species
    delete_indices = [deleted_species*2, deleted_species*2+1]
    
    # Collect complex indices involving the deleted species
    for i in range(pair_start, pair_stop, 2):
        if int(individual[i, 0, 0]) == deleted_species or int(individual[i, 0, 1]) == deleted_species:
            delete_indices.extend([i-1, i])
    print(delete_indices)
    # Delete the selected rows
    updated_individual = np.delete(individual, delete_indices, axis=0)
    
    # Update the number of species and pairs
    updated_individual[-1, -1, 0] = num_species - 1
    updated_individual[-1, -1, 1] = num_pairs - len(delete_indices)//2 + 1

    return updated_individual

def apply_species_deletion_mutation(individual, mutation_rate):
    """
    Applies a species deletion mutation to the individual with a given probability.

    This function randomly selects a species and deletes it along with all complexes involving that species 
    if a random value is below the specified mutation rate.

    Parameters:
    - individual (numpy.ndarray): The multi-dimensional array representing the species and complexes.
    - mutation_rate (float): The probability of deleting a species.

    Returns:
    - numpy.ndarray: The updated individual with the species and related complexes removed if the mutation occurred.
    """
    num_species = int(individual[-1, -1, 0])
    if np.random.rand() < mutation_rate and num_species > 1:
        deleted_species = int(np.random.choice(np.arange(1, num_species)))
        individual = species_deletion(individual=individual, deleted_species=deleted_species)

    return individual


In [245]:
ind = np.zeros((7, 10, 10))
ind[-1, -1, 0] = 2
ind[-1, -1, 1] = 1
ind[5, 0, :2] = [0, 2]
ind[-1, 0, :3] = [.2, .5, .7]
ind[-1, 2, :3] = [.2, .5, .7]
ind[-2, 1, :4] = [20, 10, 30, 80]

print(ind.shape)  # Original shape: (7, 10, 10)

ind = apply_species_insertion_mutation(ind, 0.9)
print(ind.shape)  # After insertion mutation

ind = apply_species_deletion_mutation(ind, 0.95)
print(ind.shape)  # After deletion mutation


(7, 10, 10)
(13, 10, 10)
[2, 3, 6, 7]
(9, 10, 10)
